# Machine Learning
-----


## Table of Contents
- [Introduction](#introduction)
- [Setup](#setup)
- [Problem Formulation](#model-formulation)
- [Creating Labels (Outcomes)]
- [Feature Generation](#feature-generation)
- [Model Fitting](#model-fitting)
- [Model Evaluation](#model-evaluation)
- [Machine Learning Pipeline](#machine-learning-pipeline)
- [Deployment](#deployment)
- [Exercises](#exercises)
- [Resources](#resources)

# Introduction
In this tutorial, we'll discuss how to formulate a policy problem or a social science question in the machine learning framework; how to transform raw data into something that can be fed into a model; how to build, evaluate, compare, and select models; and how to reasonably and accurately interpret model results. You'll also get hands-on experience using the `scikit-learn` package in Python with data you're familiar with from previous tutorials. 


This tutorial is based on chapter 6 of [Big Data and Social Science](https://github.com/BigDataSocialScience/).

## Setup
---
*[Back to Table of Contents](#table-of-contents)*

Before we begin, run the code cell below to initialize the libraries we'll be using in this assignment. We're already familiar with `numpy`, `pandas`, and `psycopg2` from previous tutorials. Here we'll also be using [`scikit-learn`](http://scikit-learn.org) to fit modeling.

In [ ]:
%pylab inline
import pandas as pd
import psycopg2
import sklearn
import seaborn as sns
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, 
AdaBoostClassifier)
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sqlalchemy import create_engine
sns.set_style("white")

### Global Variables

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"
schema = 'class1'
train_date = 2006
test_date = 2011
prediction_horizon = 5

### Connect to the database

In [ ]:
conn = psycopg2.connect(database=db_name, host = hostname) #database connection

# Problem Formulation
*[Go back to Table of Contents](#table-of-contents)*

  
## Our Machine Learning Problem
>Of all prisoners released, we would like to predict who is likely to reenter jail within *5* years of the day we make our prediction. For instance, say it is Jan 1, 2012 and we want to identify which 
>prisoners are likely to re-enter jail between now and end of 2016. We can run our predictive model and identify >who is most likely at risk. The is an example of a *binary classification* problem. 

Note the outcome window of 5 years is completely arbitrary. You could use a window of 5, 3, 1 years or 1 day. 

# Data Exploration and Preparation. 

In order to predict recidivism, we will be using data from the `ildoc_admit` and `ildoc_exit` table to create **labels** and **features**. 


# Building a Model

We need to munge our dataset into **features** (predictors, or dependent variables, or $X$ variables) and **labels** (independent variables, or $Y$ variables).  For ease of reference, in subsequent examples, names of variables that pertain to predictors will start with "`X_`", and names of variables that pertain to outcome variables will start with "`y_`".



## Creating Labels (or Outcome Variables)

We can write SQL code in `psql`, `dbeaver`, `pgAdmin`, or programmaticaly generate the SQL and pass to the DB using `psycopg2` to create the labels. Here is the SQL code for developing labels: 
```
-- creating labels for the training set. Of all the people (docnbr) that exited before the
-- end of 2005 get the discharge date for all prisoners release dates 

--release dates table grabs the docnbr and release year for all exists in the data
drop table if exists release_dates;
create table release_dates as
select docnbr, actdisyr, actmsryr,
case when actdisyr = 0 or actdisyr is null then actmsryr else actdisyr end release_yr
from ildoc_exit;

--grab the docnbr and release years for everyone from the years 1989-2005
--using are previous release_dates table
drop table if exists release_dates_1989_2005;
create temp table release_dates_1989_2005 as
select docnbr, release_yr
from release_dates
where release_yr between 1989 and 2005;

--there are cases where people have entered and existed prison
--multiple times, in those cases grab the last release year in 
--our time range. 
drop table if exists last_exit_1989_2005;
create temp table last_exit_1989_2005 as
select docnbr, max(release_yr) last_release_yr
from release_dates_1989_2005
group by docnbr;

--find the peple admitted from 2006 to 2010. 
drop table if exists admit_2006_2010;
create temp table admit_2006_2010 as
select docnbr, curadmyr
from ildoc_admit
where curadmyr between 2006 and 2010;

--join the exit and admit data where 0 is for no readmit
-- and 1 for readmit
drop table if exists recidivism_2005_2010;
create temp table recidivism_2005_2010 as
select r.docnbr, r.last_release_yr, a.curadmyr,
case when a.curadmyr is null then 0 else 1 end recidivism
from last_exit_1989_2005 r
left join admit_2006_2010 a on r.docnbr = a.docnbr;

-- create the labels table
drop table if exists recidivism_labels_2005_2010;
create table recidivism_labels_2005_2010 as
select distinct docnbr, recidivism
from recidivism_2005_2010;

```

In this SQL script, we first created the table `release_dates` where we took the docnbr (unique identifier) and created a new field `release_yr` which takes on the value of `actmsryr` if `actdisyr` is null or zero. Since prisoners are either released through supervised work release, `actmsryr`, or discharged directly from prison, `actdisyr`, we take the value of  `actdisyr` if it exists, or `actmsryr` if `actdisyr` doesn't exist. 

Next we created a table `release_dates_1989_2005`, which used the `release_dates` table we just created. We took all of the records for `docnbr` and `release_yr` between 1989 and 2005. Next we created a table `last_exit_1989_2005`, which takes the *maximum* (most recent) `release_yr` for every `docnbr` and writes into `last_exit_1989_2005`. This table will only have one entry per `doc_nbr`, so for any given `docnbr`, or individual, we know their *most recent* release year.

We then find everyone admitted into prison between 2006 and 2010, and do a `left join`  on the `last_exit_1989_2005` (left) table and the `recidivism_2005_2010` (right) table on the `docnbr` field. The resulting table will keep all the entries from the *left* table (most recent releases between 1989 and 2005) and add their admits between 2006 and 2010. 

In [ ]:
def create_labels(prediction_date, prediction_horizon, conn, schema='class1', overwrite=False):
    """
    Generate a list of labels and return the 
    table as a dataframe.
    
    Parameters
    ----------
    prediction_date: int
        for simplicity, we will use just the year and assume 2006 means 1/1/2006
    prediction_horizon: int
        number of years within which we want to predict if they re-enter prison.
        for example, for prediction_date = 2006 and prediction_horizon = 5, we will get everyone
        who re-entered from 1/1/2006 to 12/31/2010
    overwrite: bool
        if True runs the query if table does
        not exist
    conn: obj
        psycopg2 conection object to database
        
    Returns
    -------
    df_labels: DataFrame
        Dataframe of labels
    """
    begin_range = prediction_date
    end_range = prediction_date + prediction_horizon
    
    # check if the table you're trying to create already exists
    cursor = conn.cursor()
    query = """
            select * from information_schema.tables 
            where table_name=\'recidivism_labels_{begin_range}_{end_range}\'
            """.format(begin_range=begin_range,
                      end_range=end_range);
    cursor.execute(query) 
    
    if not(cursor.rowcount) or overwrite:
        print('generating labels')
        sql_script="""
    -- creating labels for the training set. Of all the people (docnbr) that exited before 
    --  {begin_range}

    drop table if exists {schema}.recidivism_labels_{begin_range}_{end_range}; 

    -- get the discharge date for all prisoners release dates 
    drop table if exists {schema}.release_dates;
    create table {schema}.release_dates as
    select docnbr, actdisyr, actmsryr,
    case when actdisyr = 0 or actdisyr is null then actmsryr else actdisyr end release_yr
    from ildoc.ildoc_exit;

    commit;

    --grab the docnbur and release years for everyone from the years 1989 to the day before our prediction date
    --generate labels
    drop table if exists release_dates_1989_{begin_range};
    create temp table release_dates_1989_{begin_range} as
    select docnbr, release_yr
    from {schema}.release_dates
    where release_yr >= 1989 and release_yr < {begin_range};

    commit;

    -- find the last time they exited from the previous table
    drop table if exists last_exit_1989_{begin_range};
    create temp table last_exit_1989_{begin_range} as
    select docnbr, max(release_yr) last_release_yr
    from release_dates_1989_{begin_range}
    group by docnbr;

    commit;

    -- generate a table that gives us all admits that happen between our start and end dates for labels
    drop table if exists admit_{begin_range}_{end_range};
    create temp table admit_{begin_range}_{end_range} as
    select docnbr, curadmyr
    from ildoc.ildoc_admit
    where curadmyr >= {begin_range} and  curadmyr < {end_range};

    commit;

    -- for all people released between 1989 and the date of prediction, check to see if they were admitted again 
    -- during our begin and end dates
    drop table if exists recidivism_{begin_range}_{end_range};
    create temp table recidivism_{begin_range}_{end_range} as
    select r.docnbr, r.last_release_yr, a.curadmyr,
    case when a.curadmyr is null then 0 else 1 end recidivism
    from last_exit_1989_{begin_range} r
    left join admit_{begin_range}_{end_range} a on r.docnbr = a.docnbr;

    commit;

    -- drop duplicates
    drop table if exists {schema}.recidivism_labels_{begin_range}_{end_range};
    create table {schema}.recidivism_labels_{begin_range}_{end_range} as
    select distinct docnbr, recidivism
    from recidivism_{begin_range}_{end_range};
    commit; 

    """.format(schema=schema,begin_range=begin_range,end_range=end_range)
    
        cursor.execute(sql_script)
    else:
        print('Table already generated')
    
    cursor.close()
    #df_label = pd.read_sql('select * from {schema}.recidivism_labels_{begin_range}_{end_range}'.format(
    #                                                                                schema=schema,
    #                                                                                begin_range=begin_range,
    #                                                                                end_range=end_range), conn)
    
      
    return '{schema}.recidivism_labels_{begin_range}_{end_range}'.format(schema=schema,
                                                                        begin_range=begin_range,
                                                                        end_range=end_range)

In [ ]:
# Now we can generate labels for two time periods, one starting on 2006 and another starting in 2011
train_label = create_labels(train_date, prediction_horizon, conn, schema=schema, overwrite=True)
test_label = create_labels(test_date, prediction_horizon, conn, schema=schema, overwrite=True)

Now we have a label: 0 indicates *no recidivism*, 1 indicates that person did return to jail within the outcome period (beginning of 2006 to end 2010). 

## Feature Generation
*[Go back to Table of Contents](#table-of-contents)*

Our preliminary features are the following

- `nadmits` (Aggregation): Number of times someone has been addmitted to prison between 1989-2005. The more times someone has been to prison the more times they are likely continue to be arrested. 

- `age_first_admit` (Transformation): The age someone was first admitted to prison. This is calculated by subtracting their `birth_yr` from the year they were first admitted into prison. The idea behind creating this feature is that people who are younger when they are first arrested are more likely to be arrested again. 

- `gang` (Binary): A binary (true/false) indicator of whether the person has ever been in a gang. This feature takes whatever gang membership (e.g., VL, LK BPM) and encodes affiliation as 1 and no affiliation (NULL) as 0.  

- `married` (Binary): If the `marstat` field is `M`, then `married` is 1, or else it is 0 for any other categories, such as Single (`S`), Divorced (`D`), Widowed (`W`). 

- `job` (Binary): Does the person have a job lined up when they are released from prison: `Y` (1) or `N` (0), using the field `empplanf`. 

- `vetf` (Binary): Is the individual a veteran? `Y` (1) or `N`(0).

- (`drugalcf, drugcocf, drugampf, drugmarf, drugherf, drugpcpf, drughothf, drugunkf`) (Binary): Used alcohol, cocaine, marijuana, heroin, pcp, other, unknown, respectively. For each field, `1` (positive) or `0` (negative).

- `educlvl` (Binary): `1` for graduated high school or above, `0` otherwise.(*Caution: I belive the meaning of the values of this field changed overtime*) 

- `crtfind` (Binary): `1` for habitual offender or repeated abuser of children, `0` otherwise (as determined by the court). 

SQL script to make features: 
```
-------------------------------------------------------------------------------
-- create features ------------------------------------------------------------
-------------------------------------------------------------------------------

-- number of admits
drop table if exists feature_nadmits_1989_2005;
create table feature_nadmits_1989_2005 as
select docnbr, count(*) nadmits
from ildoc_admit
where docnbr in (select docnbr from recidivism_labels_2005_2010)
and curadmyr between 1988 and 2005
group by docnbr;

--NOTE: There are about 7000 people that have no admit record. They likely entered
--into the sytem before 1989. 

-- age at first arrest
drop table if exists docnbr_admityr;

create temp table docnbr_admityr as
select docnbr, min(curadmyr) min_admityr
from ildoc_admit
group by docnbr;

-- join the table with the person birthyr columns
drop table if exists age_first_admit_birth_year;

create temp table age_first_admit_birth_year as
select da.docnbr, da.min_admityr, p.birth_year
from docnbr_admityr da
left join person p on da.docnbr = p.ildoc_docnbr;

--calculate the age at the first arrest 

drop table if exists feature_age_first_admit;

create table feature_age_first_admit as
select docnbr, (min_admityr - birth_year) age_first_admit
from age_first_admit_birth_year;

drop table if exists feature_agefirstadmit;
create table feature_agefirstadmit as
select docnbr, age_first_admit
from feature_age_first_admit
where docnbr in (select docnbr from feature_nadmits_1989_2005);

-- categorical features
drop table if exists categorical_features_1989_2005,
                     cat_features_1989_2005;

create temp table cat_features_1989_2005 as
select a.*,
 to_date(LPAD(a.actmsrdt::text,8,'0'), 'MMDDYYYY') dactmsrdt,
 to_date(LPAD(a.actdisdt::text,8,'0'), 'MMDDYYYY') dactdisdt
from ( select docnbr, gang, kids, marstat, empplanf, educlvl, vetf, crtfind1, drugalcf, drugcocf,
drugampf, drugmarf, drugherf, drugpcpf, drugothf, drugunkf, actdisyr, actdisdt,
actmsryr, actmsrdt
from ildoc_exit
where  docnbr in (select docnbr from last_exit_1989_2005)) a;

--alter table cat_features_1989_2005 add column dactmsryr date; 
--update cat_features_1989_2005 set dactmsryr = to_date(LPAD(actmsrdt::text, 8,'0'));

drop table if exists tcat_features_1989_2005;
create temp table tcat_features_1989_2005 as
select t.*,
case when t.actdisyr is null or t.actdisyr = 0 then t.dactmsrdt else t.dactdisdt end release_dt
from (select * from cat_features_1989_2005) t;


alter table tcat_features_1989_2005
drop column actdisyr,
drop column actdisdt,
drop column actmsryr,
drop column actmsrdt,
drop column dactmsrdt,
drop column dactdisdt;

drop table if exists pre_categorical_features_1989_2005;

create temp table pre_categorical_features_1989_2005 as
select g.*
from ( select t.*, rank() over (partition by docnbr order by release_dt desc)
from (select * from tcat_features_1989_2005) t ) g
where g.rank = 1;

drop table if exists raw_categorical_features_1989_2005;

create temp table raw_categorical_features_1989_2005 as
select distinct * from pre_categorical_features_1989_2005;

drop table if exists categorical_features_1989_2005;

create temp table pre_categorical_features_1989_2005 as
select g.*
from ( select t.*, rank() over (partition by docnbr order by release_dt desc)
from (select * from tcat_features_1989_2005) t ) g
where g.rank = 1;

drop table if exists raw_categorical_features_1989_2005;

create temp table raw_categorical_features_1989_2005 as
select distinct * from pre_categorical_features_1989_2005;

drop table if exists categorical_features_1989_2005;

create table categorical_features_1989_2005 as
select
        docnbr,
        case when gang is null then 0 else 1 end gang,
        case when kids > 0 then 1 else 0 end kids,
        case when marstat = 'M' then 1 else 0 end married,
        case when empplanf = 'Y' then 1 else 0 end job,
        case when educlvl in ('HS', 'GD','T1','T2','T3','T4','12','13',
        '14','15','16','17') then 1
              else 0 end educlvl,
        case when crtfind1 in ('HB', 'HCO') then 1 else 0 end crtfind,
        case when vetf = 'Y' then 1 else 0 end vetf,
        case when drugalcf in ('F','X') then 1 else 0 end drugalcf,
        case when drugcocf in ('F','X') then 1 else 0 end drugcocf,
        case when drugampf in ('F','X') then 1 else 0 end drugampf,
        case when drugmarf in ('F','X') then 1 else 0 end drugmarf,
        case when drugherf in ('F','X') then 1 else 0 end drugherf,
        case when drugpcpf in ('F','X') then 1 else 0 end drugpcpf,
        case when drugothf in ('F','X') then 1 else 0 end drugothf,
        case when drugunkf in ('F','X') then 1 else 0 end drugunkf
from raw_categorical_features_1989_2005;

-- create the age features 
drop table if exists feature_age_2005;
create table feature_age_2005 as
select ildoc_docnbr, (2005 - birth_year) age  from person
where ildoc_docnbr in (select docnbr from feature_nadmits_1989_2005);

-- join all features table together to make the feature table

drop table if exists features_1989_2005;
create table features_1989_2005 as
select  f1.docnbr,
        f1.nadmits,
        f2.age_first_admit,
        f3.gang,
        f3.kids,
        f3.married,
        f3.job,
        f3.vetf,
        f3.drugalcf,
        f3.drugcocf,
        f3.drugampf,
        f3.drugmarf,
        f3.drugherf,
        f3.drugpcpf,
        f3.drugothf,
        f3.drugunkf,
        f3.educlvl,
        f3.crtfind,
        f4.age
from feature_nadmits_1989_2005 f1
left join feature_agefirstadmit f2 on f2.docnbr = f1.docnbr
left join categorical_features_1989_2005 f3 on f1.docnbr = f3.docnbr
left join feature_age_2005 f4 on f1.docnbr = f4.ildoc_docnbr;

-- create training set to make cross-tabs with 
drop table if exists training_set_1989_2005;

create table training_set_1989_2005 as
select f1.*, r.recidivism from features_1989_2005 f1
join recidivism_labels_2005_2010 r on f1.docnbr = r.docnbr


```

In [ ]:
def create_features(prediction_date, prediction_horizon, conn, schema='class1',overwrite=False):
    """
    Generate a list of features and return the 
    table as a dataframe.
    
    Note: There has to be a table of labels that
    correspond with the same time period. 
    
    Parameters
    ----------
    prediction_date: int
        year to make prediction from
        (e.g., 2005 corresponds to Dec 31, 2005)
    conn: obj
        psycopg2 conection object to database
    overwrite: bool
        If True will run SQL script if tables
        do not exist. 
        
    Returns
    -------
    table_name: str
        name of table with features
    """
    begin_range = prediction_date
    end_range = prediction_date + prediction_horizon
        
    cursor = conn.cursor()
    query = """
            select exists(select * from information_schema.tables 
            where table_name=\'features_1989_{begin_range}\')
            """.format(begin_range=begin_range)
    cursor.execute(query)
    
    if not(cursor.rowcount) or overwrite:

    
        sql_script="""
    -------------------------------------------------------------------------------
    -- create features ------------------------------------------------------------
    -------------------------------------------------------------------------------

    -- for everyone in our labels table, we want to count number of
    -- times they've been admitted as of the prediction_date

    drop table if exists {schema}.feature_nadmits_1989_{begin_range};
    create table {schema}.feature_nadmits_1989_{begin_range} as 
    select docnbr, count(*) nadmits
    from ildoc.ildoc_admit
    where docnbr in (select docnbr from {schema}.recidivism_labels_{begin_range}_{end_range})
    and curadmyr > 1988 and curadmyr < {begin_range}
    group by docnbr;  

    commit; 
    --NOTE: There are about ### people that have no admit record. They likely entered
    --into the sytem before 1989. 

    -- age at first arrest

    -- step 1: get date of first admit
    drop table if exists docnbr_admityr;
    create temp table docnbr_admityr as
    select docnbr, min(curadmyr) min_admityr
    from ildoc.ildoc_admit
    group by docnbr;

    commit; 

    -- join the date of first admit to the table with the person birthyr columns
    drop table if exists age_first_admit_birth_year;
    create temp table age_first_admit_birth_year as
    select da.docnbr, da.min_admityr, p.birth_year
    from docnbr_admityr da
    left join {schema}.person p on da.docnbr = p.ildoc_docnbr;

    commit; 

    --now we can calculate the age at the first admit 

    drop table if exists {schema}.feature_age_first_admit; 
    create table {schema}.feature_age_first_admit as
    select docnbr, (min_admityr - birth_year) age_first_admit
    from age_first_admit_birth_year;

    commit; 

    drop table if exists {schema}.feature_agefirstadmit; 
    create table {schema}.feature_agefirstadmit as 
    select docnbr, age_first_admit
    from {schema}.feature_age_first_admit
    where docnbr in (select docnbr from {schema}.feature_nadmits_1989_{begin_range}); 

    commit; 

    -- now let's create some categorical features
    drop table if exists categorical_features_1989_{begin_range},
                     cat_features_1989_{begin_range};
    create temp table cat_features_1989_{begin_range} as
    select a.*, 
     to_date(LPAD(a.actmsrdt::text,8,'0'), 'MMDDYYYY') dactmsrdt,
     to_date(LPAD(a.actdisdt::text,8,'0'), 'MMDDYYYY') dactdisdt
    from ( select docnbr, gang, kids, marstat, empplanf, educlvl, vetf, crtfind1, drugalcf, drugcocf,
    drugampf, drugmarf, drugherf, drugpcpf, drugothf, drugunkf, actdisyr, actdisdt,
    actmsryr, actmsrdt
    from ildoc.ildoc_exit
    where  docnbr in (select docnbr from last_exit_1989_{begin_range})) a;

    commit; 



    drop table if exists tcat_features_1989_{begin_range}; 
    create temp table tcat_features_1989_{begin_range} as 
    select t.*,
    case when t.actdisyr is null or t.actdisyr = 0 then t.dactmsrdt else t.dactdisdt end release_dt
    from (select * from cat_features_1989_{begin_range}) t; 

    alter table tcat_features_1989_{begin_range} 
    drop column actdisyr,
    drop column actdisdt, 
    drop column actmsryr,
    drop column actmsrdt,
    drop column dactmsrdt,
    drop column dactdisdt; 

    commit; 

    drop table if exists pre_categorical_features_1989_{begin_range}; 
    create temp table pre_categorical_features_1989_{begin_range} as 
    select g.* 
    from ( select t.*, rank() over (partition by docnbr order by release_dt desc)
    from (select * from tcat_features_1989_{begin_range}) t ) g
    where g.rank = 1;

    commit; 

    drop table if exists raw_categorical_features_1989_{begin_range}; 
    create temp table raw_categorical_features_1989_{begin_range} as
    select distinct * from pre_categorical_features_1989_{begin_range}; 

    commit; 

    drop table if exists {schema}.categorical_features_1989_{begin_range}; 
    create table {schema}.categorical_features_1989_{begin_range} as 
    select
        docnbr,  
        case when gang is null then 0 else 1 end gang,
        case when kids > 0 then 1 else 0 end kids,
        case when marstat = 'M' then 1 else 0 end married,
        case when empplanf = 'Y' then 1 else 0 end job,
        case when educlvl in ('HS', 'GD','T1','T2','T3','T4','12','13',
        '14','15','16','17') then 1
              else 0 end educlvl,
        case when crtfind1 in ('HB', 'HCO') then 1 else 0 end crtfind,
        case when vetf = 'Y' then 1 else 0 end vetf,
        case when drugalcf in ('F','X') then 1 else 0 end drugalcf,
        case when drugcocf in ('F','X') then 1 else 0 end drugcocf,
        case when drugampf in ('F','X') then 1 else 0 end drugampf,
        case when drugmarf in ('F','X') then 1 else 0 end drugmarf,
        case when drugherf in ('F','X') then 1 else 0 end drugherf,
        case when drugpcpf in ('F','X') then 1 else 0 end drugpcpf,
        case when drugothf in ('F','X') then 1 else 0 end drugothf,
        case when drugunkf in ('F','X') then 1 else 0 end drugunkf
    from raw_categorical_features_1989_{begin_range}; 

    commit; 

    -- create the age features 
    drop table if exists {schema}.feature_age_{begin_range}; 
    create table {schema}.feature_age_{begin_range} as
    select ildoc_docnbr, ({begin_range} - birth_year) age  from {schema}.person 
    where ildoc_docnbr in (select docnbr from {schema}.feature_nadmits_1989_{begin_range});

    commit; 

    -- join all features table together to make the feature table
    drop table if exists {schema}.features_1989_{begin_range}; 
    create table {schema}.features_1989_{begin_range} as 
    select 	f1.docnbr,
            f1.nadmits,
            f2.age_first_admit,
            f3.gang,
        f3.kids,
        f3.married,
        f3.job,
        f3.vetf,
        f3.drugalcf,
        f3.drugcocf,
        f3.drugampf,
        f3.drugmarf,
        f3.drugherf,
        f3.drugpcpf,
        f3.drugothf,
        f3.drugunkf,
        f3.educlvl,
        f3.crtfind,
        f4.age 
    from {schema}.feature_nadmits_1989_{begin_range} f1
    left join {schema}.feature_agefirstadmit f2 on f2.docnbr = f1.docnbr
    left join {schema}.categorical_features_1989_{begin_range} f3 on f1.docnbr = f3.docnbr
    left join {schema}.feature_age_{begin_range} f4 on f1.docnbr = f4.ildoc_docnbr; 

    commit; 


    """.format(schema=schema, begin_range=begin_range, end_range = end_range)
    
    
        cursor.execute(sql_script)
    
    cursor.close()
    
    print('created {schema}.features_1989_{begin_range}'.format(
        schema=schema,begin_range=begin_range))
    
    table_name = '{schema}.features_1989_{begin_range}'.format(schema=schema,begin_range=begin_range)
    return table_name      

In [ ]:
train_feature_table = create_features(train_date,prediction_horizon, conn, schema=schema, overwrite=True)
test_feature_table = create_features(test_date, prediction_horizon ,conn, schema=schema, overwrite=True)

## Create Training and Validation Data Sets
*[Go back to Table of Contents](#table-of-contents)*

### Our Training Set

We are going to create a training set that will take people at the beginning of 2006 and will generate labels for them based on data from 2006-2010. The features for each person are created based on data from the beginnig of our  data (1989) up to the end of 2005.

*Note: it is important to segregate your data based on time when creating features. Otherwise there can be "leakage," where you accidentally use information that you would not have known at the time.*  This happens often when calculating aggregation features; for instance, it is quite easy to calculate an average using values that go beyond our training set time-span and not realize it.  

### Our Test (Validation) Set

We will then take the model built on that training set and validate it on the Test Set. Our testing set will use labels from 2011-2015, and our features will be generated from 1989-2010. 

### Let's now create them

In [ ]:
# generate a matrix for training the models

def create_train_or_test_matrix(feature_table_name,
                                labels_table_name, 
                                matrix_name,
                                conn,
                                schema='class1',
                                overwrite=False):
    """
    joins feature table with the labels table to generate a matrix
      
    
    Parameters
    ----------
        conn: obj
        psycopg2 conection object to database
    overwrite: bool
        If True will run SQL script if tables
        do not exist. 
        
    Returns
    -------
    table
        table with features
    """
        
    cursor = conn.cursor()
    query = """
            select * from information_schema.tables 
            where table_name=\'{matrix_name}\';
            """.format(matrix_name=matrix_name)
    cursor.execute(query)
    
    if not(cursor.rowcount) or overwrite:

        sql_script="""
    drop table if exists {schema}.{matrix_name};
    commit;
    create table {schema}.{matrix_name} as 
    select f1.*, r.recidivism from {feature_table_name} f1
    join {labels_table_name} r on f1.docnbr = r.docnbr; 
    commit; 

    """.format(matrix_name=matrix_name,
               feature_table_name=feature_table_name, 
               labels_table_name = labels_table_name,
               schema=schema)
    
    
        cursor.execute(sql_script)
    
    cursor.close()
    
    print('created {schema}.{matrix_name}'.format(
        matrix_name=matrix_name, schema=schema))
    
    return '{schema}.{matrix_name}'.format(schema=schema, matrix_name=matrix_name)


In [ ]:
train_matrix = create_train_or_test_matrix(train_feature_table,
                                            train_label,
                                            'train_matrix_{}'.format(train_date),
                                            conn,
                                            schema=schema,
                                            overwrite=True)

test_matrix = create_train_or_test_matrix(test_feature_table,
                                        test_label,
                            'test_matrix_{}'.format(test_date),
                            conn,
                            schema=schema,
                            overwrite=True)

In [ ]:
df_training = pd.read_sql('select * from {}'.format(train_matrix), conn)
df_testing = pd.read_sql('select * from {}'.format(test_matrix), conn)

In [ ]:
df_training.head()

In [ ]:
df_testing.head()

In [ ]:
isnan_training_rows = df_training.isnull().any(axis=1) # Find the rows where there are NaNs

In [ ]:
df_training[isnan_training_rows].head()

There are a number of `NaNs` in the `age_first_admit` and `age` field due to there being no `birth_yr` value in the `Person` table. Let's see how much of the data is missing these values. 

In [ ]:
nrows_training = df_training.shape[0]
nrows_training_isnan = df_training[isnan_training_rows].shape[0]

In [ ]:
print('%of frows with NaNs {} '.format(float(nrows_training_isnan)/nrows_training))

3% of our datset is not too bad, we can just drop these rows and still have plenty of data. As long as we believe that these values are *missing at random*, i.e., we think that the missing `age_first_admit` is due to a data or clerical error and not because people who are more likely to reoffend are more likely to have missing values of `age_first_admit`, it's OK to drop these rows. In other situations we can "impute" the missing data (i.e, fill in the blanks in a principled and statistically sound way).   

In [ ]:
df_training = df_training[~isnan_training_rows]

Let's check the values of the ages at first admit are reasonable.

In [ ]:
np.unique( df_training['age_first_admit'] )

Aha! It is unlikely there are one-year-old infants being booked into jail! This is likely due to an incorrect entry in the `birth_yr` in the `Person` table. On the other end of the age spectrum, the ages are more likely to be correct, but this is still something that you'd want to do a "sanity check" on with someone who knows the data well.

Let's drop any rows that have age 1.  

In [ ]:
mask = ~(df_training['age_first_admit'] == 1)
df_training = df_training[mask]

Let's check how much data we still have and how many examples of recidivism are in our training dataset. We don't necessarily need to have a perfect balance of recidivists and non-recivists, but it's good to know what the "baseline" is in our dataset.

In [ ]:
print('Number of rows: {}'.format(df_training.shape[0]))
df_training['recidivism'].value_counts(normalize=True)

We have about 200,000 examples, and about 25% of those are *positive* examples (recidivist), which is what we're trying to identify. About 75% of the examples are *negative* examples (non-recidivst).

Let's take a look at our testing set. 

In [ ]:
df_testing.head()

In [ ]:
isnan_testing_rows = df_testing.isnull().any(axis=1) # Find the rows where there are NaNs
nrows_testing = df_testing.shape[0]
nrows_testing_isnan = df_testing[isnan_testing_rows].shape[0]
print('%of rows with NaNs {} '.format(float(nrows_testing_isnan)/nrows_testing))

We see that about 2% of the rows in our testing set have missing values. This matches what we'd expect based on what we saw in the training set.

In [ ]:
df_testing[isnan_testing_rows].head()

In [ ]:
# not a good idea to do in practice so we would impute in some way
df_testing = df_testing[~isnan_testing_rows]

In [ ]:
mask = ~(df_testing['age_first_admit'] == 1)
df_testing = df_testing[mask]

In [ ]:
df_testing.head()

In [ ]:
print('Number of rows: {}'.format(df_testing.shape[0]))
df_testing['recidivism'].value_counts(normalize=True)

### Split into features and labels
Here we can decide which features/predictors to use in our model and what variable to use as the label

In [ ]:
sel_features = ['nadmits', 'age_first_admit', 'gang', 'kids', 'married', 'job', 'vetf',
               'drugalcf', 'drugcocf', 'drugampf', 'drugmarf', 'drugherf', 'drugpcpf',
               'drugothf', 'drugunkf', 'educlvl', 'crtfind', 'age']
sel_label = 'recidivism'

In [ ]:
# use conventions typically used in python scikitlearn

X_train = df_training[sel_features].values
y_train = df_training[sel_label].values
X_test = df_testing[sel_features].values
y_test = df_testing[sel_label].values

# Model Selection

## Model Evaluation 
*[Go back to Table of Contents](#table-of-contents)*

In this phase, you take the predictors from your test set and apply your model to them, then assess the quality of the model by comparing the *predicted values* to the *actual values* for each record in your testing data set. 

- **Performance Estimation**: How well will our model do once it is deployed and applied to new data?

Now let's use the model we just fit to make predictions on our test dataset, and see what our accuracy score is:

In [ ]:
# Let's fit a model
from sklearn import linear_model
model = linear_model.LogisticRegression(penalty='l1', C=1e5)
model.fit( X_train, y_train )
print(model)

When we print the model results, we see different parameters we can adjust as we refine the model based on running it against test data (values such as `intercept_scaling`, `max_iters`, `penalty`, and `solver`).  Example output:

    LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

To adjust these parameters, one would alter the call that creates the `LogisticRegression()` model instance, passing it one or more of these parameters with a value other than the default.  So, to re-fit the model with `max_iter` of 1000, `intercept_scaling` of 2, and `solver` of "lbfgs" (pulled from thin air as an example), you'd create your model as follows:

    model = LogisticRegression( max_iter = 1000, intercept_scaling = 2, solver = "lbfgs" )

The basic way to choose values for, or "tune," these parameters is the same as the way you choose a model: fit the model to your training data with a variety of parameters, and see which perform the best on the test set. An obvious drawback is that you can also *overfit* to your test set; in this case, you can alter your method of cross-validation.



## Model Understanding

Let's look at what the model learned and what the coefficients are.

In [ ]:
print "The coefficients for each of the features are " 
zip(sel_features, model.coef_[0])

In [ ]:
 
std_coef = np.std(X_test,0)*model.coef_
zip(sel_features, std_coef[0])

# Model Evaluation 

Machine learning models usually do not produce a prediction (0 or 1) directly. Rather, models produce a score (that can sometimes be interpreted a a probabilty) between 0 and 1, which lets you more finely rank all of the examples from *most likely* to *least likely* to have label 1 (positive). This score is then turned into a 0 or 1 based on a user-specified threshold. For example, you might label all examples that have a score greater than 0.5 (1/2) as positive (1), but there's no reason that has to be the cutoff. 

In [ ]:
#  from our "predictors" using the model.
y_scores = model.predict_proba(X_test)[:,1]

In [ ]:
y_scores

Let's take a look at the distribution of scores and see if it makes sense to us. 

In [ ]:
sns.distplot(y_scores, kde=False, rug=False)

Our distribution of scores is skewed, with the majority of scores on the lower end of the scale. We expect this because 75% of the data is made up of nonrecidivists, so we'd guess that a higher proportion of the examples in the test set will be negative (meaning they should have lower scores). 

In [ ]:
df_testing['y_score'] = y_scores

In [ ]:
df_testing[['docnbr', 'y_score']].head()

Tools like `sklearn` often have a default threshold of 0.5, but a good threshold is selected based on the data, model and the specific problem you are solving. As a trial run, let's set a threshold of 0.5. 

In [ ]:
calc_threshold = lambda x,y: 0 if x < y else 1 
predicted = np.array( [calc_threshold(score,0.5) for score in y_scores] )
expected = y_test

## Confusion Matrix

Once we have tuned our scores to 0 or 1 for classification, we create a *confusion matrix*, which  has four cells: true negatives, true positives, false negatives, and false positives. Each data point belongs in one of these cells, because it has both a ground truth and a predicted label. If an example was predicted to be negative and is negative, it's a true negative. If an example was predicted to be positive and is positive, it's a true positive. If an example was predicted to be negative and is positive, it's a false negative. If an example was predicted to be positive and is negative, it's a false negative.

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(expected,predicted)
print conf_matrix

The count of true negatives is `conf_matrix[0,0]`, false negatives `conf_matrix[1,0]`, true positives `conf_matrix[1,1]`, and false_positives `conf_matrix[0,1]`.

In [ ]:
# generate an accuracy score by comparing expected to predicted.
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(expected, predicted)
print( "Accuracy = " + str( accuracy ) )

We get an accuracy score of 81%. Recall that our testing dataset had 80% non-recidivists and 18% recidivists. If we had just labeled all the examples as negative and guessed non-recidivist every time, we would have had an accuracy of 80%, so our basic model is not doing much better than a "dumb classifier." That's ok, because we're just getting started!



In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(expected, predicted)
recall = recall_score(expected, predicted)
print( "Precision = " + str( precision ) )
print( "Recall= " + str(recall))

If we care about our whole precision-recall space, we can optimize for a metric known as the **area under the curve (AUC-PR)**, which is the area under the precision-recall curve. The maximum AUC-PR is 1. 

In [ ]:
def plot_precision_recall(y_true,y_score):
    """
    Plot a precision recall curve
    
    Parameters
    ----------
    y_true: ls
        ground truth labels
    y_score: ls
        score output from model
    """
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_true,y_score)
    plt.plot(recall_curve, precision_curve)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    auc_val = auc(recall_curve,precision_curve)
    print('AUC-PR: {0:1f}'.format(auc_val))
    plt.show()
    plt.clf()

In [ ]:
plot_precision_recall(expected, y_scores)

## Precision and Recall at k%

If we only care about a specific part of the precision-recall curve we can focus on more fine-grained metrics. For instance, say there is a special program for people likely to be recidivists, but only 5% can be admitted. In that case, we would want to prioritize the 5% who were *most likely* to end up back in jail, and it wouldn't matter too much how accurate we were on the 80% or so who weren't very likely to end up back in jail. 

Let's say that, out of the approximately 200,000 prisoners, we can intervene on 5% of them, or the "top" 10,000 prisoners (where "top" means highest predicted risk of recidivism). We can then focus on optimizing our **precision at 5%**.

In [ ]:
def plot_precision_recall_n(y_true, y_prob, model_name):
    """
    y_true: ls
        ls of ground truth labels
    y_prob: ls
        ls of predic proba from model
    model_name: str
        str of model name (e.g, LR_123)
    """
    from sklearn.metrics import precision_recall_curve
    y_score = y_prob
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_true, y_score)
    precision_curve = precision_curve[:-1]
    recall_curve = recall_curve[:-1]
    pct_above_per_thresh = []
    number_scored = len(y_score)
    for value in pr_thresholds:
        num_above_thresh = len(y_score[y_score>=value])
        pct_above_thresh = num_above_thresh / float(number_scored)
        pct_above_per_thresh.append(pct_above_thresh)
    pct_above_per_thresh = np.array(pct_above_per_thresh)
    plt.clf()
    fig, ax1 = plt.subplots()
    ax1.plot(pct_above_per_thresh, precision_curve, 'b')
    ax1.set_xlabel('percent of population')
    ax1.set_ylabel('precision', color='b')
    ax1.set_ylim(0,1.05)
    ax2 = ax1.twinx()
    ax2.plot(pct_above_per_thresh, recall_curve, 'r')
    ax2.set_ylabel('recall', color='r')
    ax2.set_ylim(0,1.05)
    
    name = model_name
    plt.title(name)
    plt.show()
    plt.clf()

In [ ]:
def precision_at_k(y_true, y_scores,k):
    
    threshold = np.sort(y_scores)[::-1][int(k*len(y_scores))]
    y_pred = np.asarray([1 if i >= threshold else 0 for i in y_scores ])
    return precision_score(y_true, y_pred)

In [ ]:
plot_precision_recall_n(expected,y_scores, 'LR')

In [ ]:
p_at_5 = precision_at_k(expected,y_scores, 0.01)
print('Precision at 1%: {:.2f}'.format(p_at_5))

## Machine Learning Pipeline
*[Go back to Table of Contents](#table-of-contents)*

When working on machine learning projects, it is a good idea to structure your code as a modular **pipeline**, which contains all of the steps of your analysis, from the original data source to the results that you report, along with documentation. This has many advantages:
- **Reproducibility**. It's important that your work be reproducible. This means that someone else should be able
to see what you did, follow the exact same process, and come up with the exact same results. It also means that
someone else can follow the steps you took and see what decisions you made, whether that person is a collaborator, 
a reviewer for a journal, or the agency you are working with. 
- **Ease of model evaluation and comparison**.
- **Ability to make changes.** If you receive new data and want to go through the process again, or if there are 
updates to the data you used, you can easily substitute new data and reproduce the process without starting from scratch.

# Exercise 2
*[Go back to Table of Contents](#table-of-contents)*

We have only scratched the surface of what we can do with our model. We've only tried one classifier (Logistic Regression), and there are plenty more classification algorithms in `sklearn`. Let's try them! 


In [ ]:
clfs = {'RF': RandomForestClassifier(n_estimators=500, n_jobs=-1),
       'ET': ExtraTreesClassifier(n_estimators=10, n_jobs=-1, criterion='entropy'),
        'LR': LogisticRegression(penalty='l1', C=1e5),
        'SGD':SGDClassifier(loss='log'),
        'GB': GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10),
        'NB': GaussianNB()}

In [ ]:
sel_clfs = ['RF', 'ET', 'LR', 'SGD', 'GB', 'NB']

In [ ]:
max_p_at_k = 0
for clfNM in sel_clfs:
    clf = clfs[clfNM]
    clf.fit( X_train, y_train )
    print clf
    y_score = clf.predict_proba(X_test)[:,1]
    predicted = np.array(y_score)
    expected = np.array(y_test)
    plot_precision_recall_n(expected,predicted, clfNM)
    p_at_5 = precision_at_k(expected,y_score, 0.05)
    if max_p_at_k < p_at_5:
        max_p_at_k = p_at_5
    print('Precision at 5%: {:.2f}'.format(p_at_5))

# Baseline 

It is important to check our model against a reasonable **baseline** to know how well our model is doing. Without any context, 83% accuracy can sound really great... but it's not so great when you remember that you could do almost that well by declaring everyone a non-recividist, which would be stupid (not to mention useless) model. 

A good place to start is checking against a *random* baseline, assigning every example a label (positive or negative) completely at random. 

In [ ]:
max_p_at_k

In [ ]:
random_score = [random.uniform(0,1) for i in enumerate(y_test)] 
random_predicted = np.array( [calc_threshold(score,0.5) for score in random_score] )
random_p_at_5 = precision_at_k(expected,random_predicted, 0.05)

Another good practice is checking against an "expert" or rule of thumb baseline. For example, say that talking to people at the DOC, you find that they think it's much more likely that someone who has been in prison multiple times already will reoffend. Then you should check that your classifier does better than just labeling everyone who has had multiple past admits as positive.

In [ ]:
recidivism_predicted = np.array([ 1 if nadmit > 1 else 0 for nadmit in df_testing.nadmits.values ])
recidivism_p_at_5 = precision_at_k(expected,recidivism_predicted,0.05)

In [ ]:
all_non_recidivist = np.array([0 for nadmit in df_testing.nadmits.values])
all_non_recidivist_p_at_5 = precision_at_k(expected, all_non_recidivist,0.05)

In [ ]:
sns.set_style("whitegrid")
fig, ax = plt.subplots(1, figsize=(22,12))
sns.set_context("poster", font_scale=1.25, rc={"lines.linewidth":2.25, "lines.markersize":8})
sns.barplot(['Random','All Non-Recidivist', 'Recidivism','Model'],
            [random_p_at_5, all_non_recidivist_p_at_5, recidivism_p_at_5, max_p_at_k],
            palette=['#6F777D','#6F777D','#6F777D','#800000'])
plt.ylim(0,1)
plt.ylabel('precision at 5%')

## Let's explore some of the models we just built

In [ ]:
clfs

In [ ]:
# explore random forest RF
sel_clfs
clf = clfs[sel_clfs[0]]
#clf = clfs[clfNM]
print clf
clf.fit( X_train, y_train )
print clf.feature_importances_

### Let's see if we can make this look a little better

In [ ]:
importances = clf.feature_importances_
std = np.std ([tree.feature_importances_ for tree in clf.estimators_],
       axis=0)
indices = np.argsort(importances)[::-1]

print ("Feature ranking")
for f in range(X_test.shape[1]):
    print ("%d. %s (%f)" % (f + 1, sel_features[f], importances[indices[f]]))

# plot 
plt.figure
plt.title ("Feature Importances")
plt.bar(range(X_test.shape[1]), importances[indices], color='r',
      yerr=std[indices], align = "center")
plt.xticks(range(X_test.shape[1]), sel_features, rotation=90)
plt.xlim([-1, X_test.shape[1]])
plt.show




# Exercise 

Our model has just scratched the surface. Try the following: 
    
- Create more features
- Try more models
- Try different parameters for your model

## Resources
*[Go back to Table of Contents](#table-of-contents)*

- Hastie et al.'s [The Elements of Statistical Learning](http://statweb.stanford.edu/~tibs/ElemStatLearn/) is a classic and is available online for free.
- James et al.'s [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/), also available online, includes less mathematics and is more approachable.
- Wu et al.'s [Top 10 Algorithms in Data Mining](http://www.cs.uvm.edu/~icdm/algorithms/10Algorithms-08.pdf).